# Tutorial 1. Param

:::{note}
:icon: false

#### Param basics with `panel-material-ui` widgets

[Param](https://param.holoviz.org/en/docs/latest/) is the foundation for reactivity in Panel. It gives you:

- validated state (`Parameter`)
- reusable state containers (`Parameterized`)
- declarative dependencies (`@param.depends`, `pn.bind`, `pn.rx`)

In this tutorial we focus on Param concepts, and use `panel_material_ui` widgets for interactive controls.

## Learning goals

By the end, you should be able to:

- explain the difference between a parameter value and a `Parameter` object
- use parameter references to drive reactive UI
- build reusable class-based components powered by Param
:::

In [ ]:
import pandas as pd
import panel as pn
import panel_material_ui as pmui
import param

pn.extension("tabulator")

## 1) `Parameter` and `Parameterized`

`param.Parameterized` is a class that owns parameters.  

A `param.Parameter` describes the semantics of a value: type, bounds, defaults, docs, and validation.

To make this a bit more concrete, let’s start by building a simple calculator with Param.


In [ ]:
class Adder(param.Parameterized):
    left = param.Number(default=0)
    right = param.Number(default=0)
    result = param.Number(default=0)

Adder(left=2)

The calculator is not updating the `result`. We need to wire it up by using `param.depends` on a function which calculates the sum. Instead of rewriting the class we inherit the baseclass:

In [ ]:
class DynamicAdder(Adder):

    @param.depends("left", "right", watch=True, on_init=True)
    def _calculate_result(self):
        self.result = self.left + self.right
        print(f"The result is: {self.result}")

adder = DynamicAdder()

We can also change the input attributes directly and see the updated result:

In [ ]:
adder.left = 2

Updating `left` and `right` would trigger the callback twice. So we should generally update all parameters at once:

In [ ]:
adder.param.update(left=3, right=4);

Finally, we can make it a bit more advanced by supporting different operations:

In [ ]:
import operator

operations = {"+": operator.add, "-": operator.sub, "*": operator.mul, "/": operator.truediv}

class Calculator(param.Parameterized):
    left = param.Number(default=1)
    right = param.Number(default=1)
    op = param.Selector(default="+", objects=["+", "-", "*", "/"])
    result = param.Number(default=0, constant=True)

    @param.depends("left", "right", "op", watch=True, on_init=True)
    def _calculate_result(self):
        with param.edit_constant(self):
            self.result = operations[self.op](self.left, self.right)

calc = Calculator(left=3, right=4, op="*")

calc.result

Using Panel we can render the the whole thing as interactive widgets:

In [ ]:
pn.Param(calc)

## 2) Value vs parameter object

This distinction is crucial:

- `widget.value` -> current plain Python value
- `widget.param.value` -> the `Parameter` object (reactive reference)

In [ ]:
name = pmui.TextInput(label="Name", value="Ada")

name.value         # e.g. "Ada"
name.param.value   # Parameter reference

Use references to declare reactive outputs:

In [ ]:
greeting = pn.bind(lambda v: f"## Hello {v or 'there'}!", name.param.value)

pn.Column(name, pn.pane.Markdown(greeting))

## 3) Parameters as references

You can pass parameter references directly to other components:

In [ ]:
class CardState(param.Parameterized):
    title = param.String(default="Reactive card")
    visible = param.Boolean(default=True)

state = CardState()

controls = pmui.Column(
    pmui.TextInput.from_param(state.param.title, label="Card title"),
    pmui.Switch.from_param(state.param.visible, label="Visible"),
)

card = pmui.Card(
    "This card is driven by Param references.",
    title=state.param.title,
    visible=state.param.visible,
)

pn.Row(controls, card)

## 4) Transforming values with `pn.rx`

`pn.rx` lets you write expression-style pipelines that stay reactive.

In [ ]:
name = pmui.TextInput(label="Name", value="World")
color = pmui.Select(label="Color", options=["teal", "purple", "tomato"], value="teal")

text = pn.rx("# Hello {}").format(name.param.value)
styles = {"color": color.param.value}

pn.Column(name, color, pn.pane.Markdown(text, styles=styles))

### Mini Exercise 1

Create a small app where a slider controls the `font-size` of a Markdown pane.

In [ ]:
size = pmui.FloatSlider(name="Font size", start=8, end=36, step=1, value=14)
md = pn.pane.Markdown("Reactive typography")

pn.Row(size, md)

:::{note} Hint
:class: dropdown

`styles["font-size"]` expects a string such as `"14px"`.

:::

:::{note} Solution
:class: dropdown

```python
size = pmui.FloatSlider(name="Font size", start=8, end=36, step=1, value=14)
font_size = pn.rx("{}px").format(size.param.value)
md = pn.pane.Markdown("Reactive typography", styles={"font-size": font_size})

pn.Row(size, md)
```
:::


## 5) Building reusable class-based components

For larger apps, keep your state in a class and render via `Viewer`.

In [ ]:
from panel.viewable import Viewer

class DataExplorer(Viewer):
    data = param.DataFrame(doc="Data to explore")
    page_size = param.Integer(default=10, bounds=(1, 50), step=5)
    show_index = param.Boolean(default=True)
    compact = param.Boolean(default=False, label="Compact layout")

    def __init__(self, **params):
        super().__init__(**params)
        self._table = pn.widgets.Tabulator(
            self.param.data,
            page_size=self.param.page_size,
            show_index=self.param.show_index,
            sizing_mode="stretch_width",
            height=360,
        )

    @param.depends("compact")
    def __panel__(self):
        controls = pmui.Column(
            pmui.FloatSlider.from_param(self.param.page_size, start=5),
            pmui.Switch.from_param(self.param.show_index),
            pmui.Switch.from_param(self.param.compact),
        )
        if self.compact:
            return pn.Column(controls, self._table)
        return pn.Row(controls, self._table, sizing_mode="stretch_width")

DataExplorer(data=pd.read_parquet("./windturbines.parq"))

### Section Recap Exercise

Extend `DataExplorer` with:

1. `manufacturer` filter (`param.Selector`)
2. filtered table output showing only the selected manufacturer

:::{note} Hint
:class: dropdown

- Populate selector options in `__init__` from `self.data["t_manu"]`.
- Add `filtered()` with `@param.depends("data", "manufacturer")`.
- Render `self.filtered()` in `Tabulator`.

:::

:::{note} Solution
:class: dropdown

```python
class DataExplorer(Viewer):
    data = param.DataFrame(doc="Data to explore")
    page_size = param.Integer(default=10, bounds=(1, 50))
    show_index = param.Boolean(default=True)
    manufacturer = param.Selector(default=None, objects=[])

    def __init__(self, **params):
        super().__init__(**params)
        options = sorted(self.data["t_manu"].dropna().unique().tolist())
        self.param.manufacturer.objects = [None, *options]
        self.manufacturer = None

    @param.depends("data", "manufacturer")
    def filtered(self):
        if self.manufacturer is None:
            return self.data
        return self.data[self.data["t_manu"] == self.manufacturer]

    @param.depends("data", "manufacturer", "page_size", "show_index")
    def table(self):
        return pn.widgets.Tabulator(
            self.filtered(),
            page_size=self.page_size,
            show_index=self.show_index,
            sizing_mode="stretch_width",
            height=360,
        )

    def __panel__(self):
        controls = pmui.Column(
            pmui.Select.from_param(self.param.manufacturer, label="Manufacturer"),
            pmui.FloatSlider.from_param(
                self.param.page_size,
                name="Page size",
                start=5,
                end=40,
                step=5,
            ),
            pmui.Switch.from_param(self.param.show_index, label="Show index"),
            sizing_mode="stretch_width",
        )
        return pn.Row(controls, self.table, sizing_mode="stretch_width")
```
:::


## Recap

You now have the essentials:

- define validated reactive state with `Parameter`s
- differentiate value access (`.value`) from reactive references (`.param.value`)
- transform references declaratively with `pn.bind` and `pn.rx`
- package state + UI in reusable class-based components

This is the foundation for the rest of the tutorial sequence.